# Preprocesamiento de datos

## Ejercicios: Parte I

<div class="alert-warning">
    
Indicaciones: Los ejercicios 1, 2, 3 y 4 deben leer su data de links en Github y el código y los resultados deben ser publicados en páginas en Github.

</div>

## Ejercicio 1: Preprocesamiento de mapas de plantas de energía solar y eólica en China

<div class="alert-success">
    
1. Use two maps of points.

2. Compute the distance matrix for both maps.

3. Select one row of the distance matrix, and plot the two points with the minimal distance on top of the country of your choosing. (distancia entre punto a punto)
</div>

Mapas de puntos: Plantas de energía solar y eólica en China

Fuente: [Global Power Plant Database](https://datasets.wri.org/dataset/globalpowerplantdatabase)

In [187]:
import pandas as pd
import geopandas as gpd
import os

PowerPlants = pd.read_csv("https://github.com/Ciencia-de-datos-espaciales-2023-2/Evaluacion4_1ra-Parte_Mining_GeoDFs/raw/main/data/global_power_plant_database.csv", dtype={'other_fuel3': 'object'})

China_PowerPlants = PowerPlants[PowerPlants["country"]=="CHN"]
China_PowerPlants = China_PowerPlants.loc[:,["name","gppd_idnr","capacity_mw","latitude","longitude","primary_fuel"]]
China_PowerPlants.reset_index(drop=True, inplace=True)

#Dataframe a GeoDataframe:
China_PowerPlants_gpd = gpd.GeoDataFrame(data=China_PowerPlants.copy(),
                                        geometry=gpd.points_from_xy(x=China_PowerPlants.longitude,
                                        y=China_PowerPlants.latitude),
                                        crs=4326)

#Cambiando los CRSs por unos proyectados y que coincidan con los de China3415:
China3415_PowerPlants = China_PowerPlants_gpd.to_crs(3415)

#Mapas de puntos: Plantas de energía solar y eólica
China3415_SolarPowerPlants = China3415_PowerPlants[China3415_PowerPlants.primary_fuel=="Solar"]
China3415_SolarPowerPlants = China3415_SolarPowerPlants.rename(columns={"name":"SolarPowerPlant_Name"})
China3415_SolarPowerPlants = China3415_SolarPowerPlants.reset_index()
China3415_SolarPowerPlants.drop(columns="index",inplace=True)


China3415_WindPowerPlants = China3415_PowerPlants[China3415_PowerPlants.primary_fuel=="Wind"]
China3415_WindPowerPlants = China3415_WindPowerPlants.rename(columns={"name":"WindPowerPlant_Name"})
China3415_WindPowerPlants = China3415_WindPowerPlants.reset_index()
China3415_WindPowerPlants.drop(columns="index",inplace=True)


# Geopackage con los 2 mapas de puntos:
China3415_SolarPowerPlants.to_file(os.path.join("maps","China_3415","China3415_PowerPlant.gpkg"), layer="Solar", driver="GPKG",index="")
China3415_WindPowerPlants.to_file(os.path.join("maps","China_3415","China3415_PowerPlant.gpkg"), layer="Wind", driver="GPKG",index="")

 ## Ejercicio 2: Preprocesamiento de mapas de plantas de energía nuclear y de arroyos en China

<div class="alert-success">
    
1. Use a map of points and a map of lines.

2. Compute the distance matrix for both maps.

3. Select one line of the distance matrix, and plot the closests and the farthest point to that line. (Se puede seleccionar un río o un sistema y calcular el punto más cercano o lejano a la línea. Si se escoge un punto, se calcula la línea más cercana y lejana)
    
    
</div>

Plantas de energía nuclear en China:

In [188]:
China3415_NuclearPowerPlants = China3415_PowerPlants[China3415_PowerPlants.primary_fuel=="Nuclear"]
China3415_NuclearPowerPlants = China3415_NuclearPowerPlants.rename(columns={"name":"NuclearPowerPlant_Name"})
China3415_NuclearPowerPlants = China3415_NuclearPowerPlants.reset_index()
China3415_NuclearPowerPlants.drop(columns="index",inplace=True)

China3415_NuclearPowerPlants.to_file(os.path.join("maps","China_3415","China3415_PowerPlant.gpkg"), layer="Nuclear", driver="GPKG",index="")

Mapa de China: [World_Countries](https://www.efrainmaps.es/english-version/free-downloads/world/)

In [183]:
#Mapa de China
countries = gpd.read_file("https://github.com/Ciencia-de-datos-espaciales-2023-2/Evaluacion4_1ra-Parte_Mining_GeoDFs/raw/main/maps/World_Countries/World_Countries.shp")
China = countries[countries.COUNTRY=='China']
China3415_country = China.to_crs(3415)
China3415_country = China3415_country.reset_index()
China3415_country.drop(columns="index",inplace=True)
China3415_country.to_file(os.path.join("maps","China_3415","China3415_GeoMap.gpkg"), layer="Country", driver="GPKG",index="")

Mapa de líneas: Mapas de arroyos al sur y este de Asia, [Stream network dataset](https://data.apps.fao.org/catalog/iso/dc2a5121-0b32-482b-bd9b-64f7a414fa0d)

In [184]:
#Mapa de arroyos al sureste de Asia
AsiaStreamLink= "https://github.com/Ciencia-de-datos-espaciales-2023-2/Evaluacion4_1ra-Parte_Mining_GeoDFs/raw/main/maps/Asia_Streams/rivers_asia_37331.shp"
AsiaStream = gpd.read_file(AsiaStreamLink)
AsiaStream_3415 = AsiaStream.to_crs(3415)

#Filtrando los arroyos para solo englobar aquellos dentro de China
China3415_Streams = gpd.clip(gdf = AsiaStreams_3415 , mask = China3415_country)
China3415_Streams = China3415_Streams.reset_index()
China3415_Streams.drop(columns=["index"],inplace=True)

byecol = set(China3415_Streams)
China3415_Streams["Stream_UniqueArcIdentifier/ARCID"] =China3415_Streams["ARCID"]
China3415_Streams["Major_Basin"] =China3415_Streams["MAJ_NAME"]
China3415_Streams["Sub_Basin"] =China3415_Streams["SUB_NAME"]
byecol = set(China3415_Streams.columns)-{"Stream_UniqueArcIdentifier_ARCID","Major_Basin","Sub_Basin","geometry"}

China3415_Streams.drop(columns=byecol, inplace=True)
China3415_Streams.to_file(os.path.join("maps","China_3415","China3415_GeoMap.gpkg"), layer="Streams", driver="GPKG",index="")

## Ejercicios 3 y 4: Preprocesamiento de mapas de ríos agrupados por cuenca principal y convex hulls de ríos en una misma cuenca principal

<div class="alert-success">
    
1. Create a set of points and a set of polygons

2. Compute the distance matrix for both sets.

3. Select one polygon of the distance matrix, and plot the closests and the farthest point to that polygon.
</div>   

<div class="alert-success">
    
1. Create a set of points and a set of lines

2. Get the buffer for the lines, select different values for the distance. Jugar con varias distancias, pero solo colocar una que tenga resultados interesantes en el notebook final.

3. Keep the points that are within the buffer (as in point 2, you need to play with differn distances until you show something interesting. Todos los puntos que están dentro del buffer creado para la línea.
   

</div>   

Conjunto de líneas para el ejercicio 4: Ríos agrupados por cuenca principal (Major_Basin) 

In [185]:
#GeoDataFrame, agrupando los ríos por cuenca principal (Major_Basin)
China3415_MajorStreamBasin = China3415_Streams.dissolve(by="Major_Basin")
China3415_MajorStreamBasin = China3415_MajorStreamBasin.drop(columns = ["Stream_UniqueArcIdentifier_ARCID","Sub_Basin"])
China3415_MajorStreamBasin = China3415_MajorStreamBasin.reset_index()

China3415_MajorStreamBasin.to_file(os.path.join("maps","China_3415","China3415_GeoMaps.gpkg"), layer="MajorStream_Basins", driver="GPKG",index="")

Conjunto de polígonos para el ejercicio 3: Convex hulls de ríos agrupados por cuenca principal 

In [186]:
#Creando convex hulls que engloblen los ríos de cada cuenca principal para obtener un GeoDataFrame con polígonos
China3415_MajorRiverBasin_Hulls = China3415_MajorRiverBasin.convex_hull.to_frame()

Major_Basins = China3415_MajorRiverBasin.Major_Basin.tolist()
China3415_MajorRiverBasin_Hulls["Major_Basin"]= Major_Basins
China3415_MajorRiverBasin_Hulls.rename(columns = {0:"geometry"}, inplace = True)
China3415_MajorRiverBasin_Hulls = China3415_MajorRiverBasin_Hulls.set_geometry("geometry")

China3415_MajorRiverBasin_Hulls.to_file(os.path.join("maps","China_3415","China3415_GeoMap.gpkg"), layer="MajorRiver_BasinsHulls", driver="GPKG",index="")